##### COORESPONDANCE DES FICHIERS COLLECTES

In [15]:
import pandas as pd
from datetime import datetime, timedelta
import os, sys
import numpy as np
import re
from sqlalchemy import create_engine
import logging

In [17]:
panier_df = pd.read_excel("Echantillon_Data_Scrapping_09032024_valide_code.xlsx")
panier_df.head()

len(panier_df)

1458

##### COORESPONDANCE AVEC LE PANIER DE BIEN

In [19]:
colonnes_a_comparer = ["Libelle_du_produit"] 

date_debut = datetime.strptime("01-04-2024", "%d-%m-%Y")  
date_fin = datetime.strptime("30-04-2024", "%d-%m-%Y")  
# Traiter les intervalles de prix pour obtenir le plus petit prix
def extract_min_price(value):
    """
    Extraire la valeur de gauche dans l'interval.
    """
    if isinstance(value, str):
        if "-" in value:
            parts = value.split("-")
            if len(parts) >= 2:
                return int(parts[0].strip())
        elif " " in value:
            return int(value.split()[0])
    return value

def extraction_nom_site(url):
    """
    Extrait le nom du site d'une URL complète.
    """
    sites = ['auchan', 'kevajo', 'ivoirshop']
    
    for site in sites:
        if site in url:
            return site
        
    return None

while date_debut <= date_fin:
    date_str = date_debut.strftime("%d%m%Y")
    fichier_journalier = os.path.join("Data_Collecte","Avril", f"Data_Scrapping_{date_str}.xlsx")
    try:
        df_journalier = pd.read_excel(fichier_journalier)
        
        lignes_correspondantes = pd.merge(df_journalier, panier_df[['Libelle_du_produit', 'Code produit', 'Code']], how="inner", on=colonnes_a_comparer)

        lignes_correspondantes['Code_site'] = lignes_correspondantes['Code_site'].apply(lambda x: extraction_nom_site(str(x)) if pd.notnull(x) and x != 'adjovan' else x)
        lignes_correspondantes['Code_site'].fillna('adjovan', inplace=True)

        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].fillna("")
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: x.replace(".", "") if isinstance(x, str) else x)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: x.replace(",", "") if isinstance(x, str) else x)
        #lignes_correspondantes["Prix_du_produit"] = pd.to_numeric(lignes_correspondantes["Prix_du_produit"].str.replace("[^\d-]", "", regex=True), errors='coerce')
        #lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(lambda x: float(x.split("-")[0]) if isinstance(x, str) and "-" in x else x)
        lignes_correspondantes["Prix_du_produit"] = lignes_correspondantes["Prix_du_produit"].apply(extract_min_price)
        
        fichier_sortie = os.path.join("Data_Valide_Prix","avril",f"Data_Scrapping_{date_str}.xlsx")
        lignes_correspondantes.to_excel(fichier_sortie, index=False)
        
    except FileNotFoundError:
        print(f"Le fichier {fichier_journalier} n'existe pas.")
 
    date_debut += timedelta(days=1)

Le fichier Data_Collecte\Avril\Data_Scrapping_01042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_07042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_11042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_12042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_14042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_17042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_21042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_22042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_23042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_26042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_27042024.xlsx n'existe pas.
Le fichier Data_Collecte\Avril\Data_Scrapping_29042024.xlsx n'existe pas.


##### FUSION ET AGGREGATION DES DONNEES DU MOIS PRECEDENT(Mars)

In [22]:
#AJOUT DES TYPES DE VARIETE AUX COLLECTES JOURNALIERES
panier_df2 = pd.read_excel("Panier_renomme.xlsx")
def clean_and_convert(value):
    if pd.isna(value):
        return np.nan
    elif isinstance(value, (int, float)):
        return int(value)  
    elif isinstance(value, str):
        try:
            numeric_part = re.search(r'\d+', value).group()  
            return int(numeric_part) 
        except (TypeError, AttributeError, ValueError):
            return np.nan  
'''  
def mean_ignore_nan(values):
    if values.isnull().all():
        return np.nan
    return np.nanmean(values)
'''

date_debut = datetime.strptime("01-04-2024", "%d-%m-%Y")  
date_fin = datetime.strptime("30-04-2024", "%d-%m-%Y")

while date_debut <= date_fin:
    date_str = date_debut.strftime("%d%m%Y")
    fichier_journalier = os.path.join("Data_Valide_Prix","avril", f"Data_Scrapping_{date_str}.xlsx")
    try:
        df_journalier = pd.read_excel(fichier_journalier)
        
        df_journalier['Prix_du_produit'] = df_journalier['Prix_du_produit'].apply(clean_and_convert)
        df_journalier['Unite_monetaire'] ='CFA'
        #df_journalier['Date_de_collecte'] = pd.to_datetime(df_journalier['Date_de_collecte'])
        premiere_date = df_journalier.loc[0, 'Date_de_collecte'].split()[0]
        df_journalier['Date_de_collecte'] = premiere_date
        #donnees_groupees = df_journalier.groupby(['Date_de_collecte','Unite_monetaire','Libelle_du_produit']).agg({'Prix_du_produit': 'mean'}).reset_index()
        donnees_groupees2 = pd.merge(df_journalier, panier_df2[['Code produit','Type variété (HE, O1,O2,O3)']], how="inner", on="Code produit")
        #donnees_groupees = df_journalier.groupby(['Date_de_collecte', 'Unite_monetaire', 'Code produit']).agg({'Prix_du_produit': mean_ignore_nan}).reset_index()
        #print(f"Data_Scrapping_{date_str}.xlsx")
        fichier_sortie = os.path.join("Data_correction1","avril",f"Data_Scrapping_{date_str}.xlsx")
        donnees_groupees2.to_excel(fichier_sortie, index=False)
            
    except FileNotFoundError:
        print(f"Le fichier {fichier_journalier} n'existe pas.")
 
    date_debut += timedelta(days=1)

Le fichier Data_Valide_Prix\avril\Data_Scrapping_01042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_07042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_11042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_12042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_14042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_17042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_21042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_22042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_23042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_26042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_27042024.xlsx n'existe pas.
Le fichier Data_Valide_Prix\avril\Data_Scrapping_29042024.xlsx n'existe pas.


In [23]:
#FUSION DE TOUS LES FICHIERS DU MOIS PRECEDENT
dossier = os.path.join("Data_correction1","mars")
dfs = []

for fichier in os.listdir(dossier):
        
        chemin_complet = os.path.join(dossier, fichier)
        df = pd.read_excel(chemin_complet)
        dfs.append(df)

df_final = pd.concat(dfs, ignore_index=True)

path_out = os.path.join("Data_correction1","mars","F_Data_Scrapping_Mars.xlsx")
df_final.to_excel(path_out, index=False)


In [34]:
#Interchangement des colonnes "Date_de_collecte" et "Code_site"
path_out1 = os.path.join("Data_correction1","mars","F_Data_Scrapping_Mars.xlsx")
df = pd.read_excel(path_out1)
columns1 = list(df.columns)
columns1[columns1.index('Date_de_collecte')], columns1[columns1.index('Code_site')] = columns1[columns1.index('Code_site')], columns1[columns1.index('Date_de_collecte')]
df = df[columns1]

,Date_de_collecte,Code_site,Libelle_du_produit,Quantite,Prix_du_produit,Caracteristique,Unite,Unite_monetaire,Code produit,Code,"Type variété (HE, O1,O2,O3)"
0,2024-03-01,ivoirshop,Thé Homme Fort – Performance Au Lit – Virilité...,Status:In stock,7500.0,NaN,NaN,CFA,1020100030701,111111,Homogene
1,2024-03-01,ivoirshop,Thé Homme Fort – Performance Au Lit – Virilité...,Status:In stock,7500.0,NaN,NaN,CFA,1020100030701,111111,Homogene
2,2024-03-01,ivoirshop,Thé Homme Fort – Performance Au Lit – Virilité...,Status:In stock,7500.0,NaN,NaN,CFA,1020100030701,111111,Homogene
3,2024-03-01,ivoirshop,Thé Homme Fort – Performance Au Lit – Virilité...,Status:In stock,7500.0,NaN,NaN,CFA,1020100030701,111111,Homogene
4,2024-03-01,ivoirshop,1 Paquet Thé victoria – Moringa – 25 Sachets,Status:4 en stock,NaN,NaN,NaN,CFA,1020100030701,111111,Homogene


In [37]:
#AGGREGATION DES DONNEES DU FICHIER DU MOIS PAR PRIX ET AJOUT DE LA COLONNE "FOURCHETTE".

# Calculer les fourchettes pour chaque produit
def calculate_price_ranges(row):
    price = row['Prix_du_produit']
    
    if row['Type variété (HE, O1,O2,O3)'] == 'Heterogene':
        lower_bound = price * 0.85
        upper_bound = price * 1.15
    elif row['Type variété (HE, O1,O2,O3)'] == 'Homogene':
        lower_bound = price * 0.70
        upper_bound = price * 1.30
    else:
        raise ValueError(f"Variété non reconnue pour le produit {row['Libelle_du_produit']}")
    
    return f"{lower_bound:.2f}-{upper_bound:.2f}"

path1 = os.path.join("Data_correction1","mars","F_Data_Scrapping_Mars.xlsx")
df_FDSM = pd.read_excel(path1)
df_FDSM['Unite_monetaire'] ='CFA'
premiere_date = df_FDSM.loc[0, 'Date_de_collecte'].split()[0]
df_FDSM['Date_de_collecte'] = premiere_date
df_FDSM_A = df_FDSM.groupby(['Date_de_collecte','Code_site','Libelle_du_produit','Unite_monetaire','Type variété (HE, O1,O2,O3)']).agg({'Prix_du_produit': 'mean'}).reset_index()
df_FDSM_A['fourchette'] =df_FDSM_A.apply(calculate_price_ranges, axis=1)
path_out = os.path.join("Data_correction1","mars","A_Data_Scrapping_Avril.xlsx")
df_FDSM_A.to_excel(path_out, index=False)

##### CORRECTION 

##### Control sur les prix selon la fourchette attribuée à la variété

In [11]:
previous_month_file = os.path.join("Data_correction1","mars","A_Data_Scrapping_Avril.xlsx")

aggregated_df = pd.read_excel(previous_month_file)

# Créer un dictionnaire pour un accès rapide aux fourchettes
price_ranges = {}

for _, row in aggregated_df.iterrows():
    lower_bound, upper_bound = map(float, row['fourchette'].split('-')) 
    price_ranges[row['Libelle_du_produit']] = (lower_bound, upper_bound)

# Fonction pour vérifier si le prix est dans la fourchette
def is_within_range(product, price):
    if product not in price_ranges:
        return None
    lower_bound, upper_bound = price_ranges[product]
    return lower_bound <= price <= upper_bound

daily_files_directory= os.path.join("Data_correction1","avril")

# Parcourir chaque fichier journalier
for daily_file in os.listdir(daily_files_directory):
    if daily_file.endswith('.xlsx') : 
        daily_file_path = os.path.join(daily_files_directory, daily_file)
        daily_df = pd.read_excel(daily_file_path)
        daily_df['status'] = None
        daily_df['Valeur_Atypique'] = None

        # Créer une liste des lignes à conserver
        rows_to_keep = []

        for _, row in daily_df.iterrows():
            product = row['Libelle_du_produit']
            if product in price_ranges:
                price = row['Prix_du_produit']
                if pd.notna(price) and not is_within_range(product, price):
                    row['Prix_du_produit'] = None
                rows_to_keep.append(row)
            else:
                # Produit non présent dans le fichier du mois précédent
                continue

        # Convertir la liste des lignes à conserver en DataFrame
        cleaned_df = pd.DataFrame(rows_to_keep, columns=daily_df.columns)

        cleaned_file_path = os.path.join("Data_Correction2","avril",daily_file)
        cleaned_df.to_excel(cleaned_file_path, index=False)


In [39]:
#MISE A JOUR DU PROGRAMME DE CORRECTION DES PRIX
previous_month_file = os.path.join("Data_correction1","mars","A_Data_Scrapping_Avril.xlsx")

aggregated_df = pd.read_excel(previous_month_file)

# Créer un dictionnaire pour un accès rapide aux fourchettes
price_ranges = {}

for _, row in aggregated_df.iterrows():
    lower_bound, upper_bound = map(float, row['fourchette'].split('-')) 
    price_ranges[row['Libelle_du_produit']] = (lower_bound, upper_bound)

# Fonction pour vérifier si le prix est dans la fourchette
def is_within_range(product, price):
    if product not in price_ranges:
        return None
    lower_bound, upper_bound = price_ranges[product]
    return lower_bound <= price <= upper_bound

daily_files_directory= os.path.join("Data_correction1","avril")

# Parcourir chaque fichier journalier
for daily_file in os.listdir(daily_files_directory):
    if daily_file.endswith('.xlsx') : 
        daily_file_path = os.path.join(daily_files_directory, daily_file)
        daily_df = pd.read_excel(daily_file_path)
        daily_df['Status'] = None
        daily_df['Valeur_Atypique'] = None

        # Créer une liste des lignes à conserver
        rows_to_keep = []

        for _, row in daily_df.iterrows():
            product = row['Libelle_du_produit']
            if product in price_ranges:
                price = row['Prix_du_produit']
                if pd.isna(price):
                    row['Status'] = 'Missing'
                elif not is_within_range(product, price):
                    row['Status'] = 'Atypique'
                    row['Valeur_Atypique'] = f"Valeur atypique:{price}"
                    row['Prix_du_produit'] = None 
                else:
                    row['Status'] = 'Valide'
                rows_to_keep.append(row)
            else:
                # Produit non présent dans le fichier du mois précédent
                continue

        cleaned_df = pd.DataFrame(rows_to_keep, columns=daily_df.columns)

        cleaned_file_path = os.path.join("Data_Correction2","avril",daily_file)
        cleaned_df.to_excel(cleaned_file_path, index=False)


##### TRIATEMENT D'IMPUTATION DES PRIX MANQUANTS

In [42]:
#MISE A JOUR DU PROGRAMME D'IMPUTATION DES PRIX
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levellevel)s - %(message)s')

fichier_mois_precedent = os.path.join("Data_correction1", "mars", "A_Data_Scrapping_Avril.xlsx")
df_mois_precedent = pd.read_excel(fichier_mois_precedent)

logging.info("Fichier agrégé du mois précédent chargé.")

# Fonction pour l'imputation des prix manquants
def imputer_prix_manquants(df_journalier, df_mois_precedent):
    for index, row in df_journalier.iterrows():
        if pd.isna(row["Prix_du_produit"]):
            produit = row["Libelle_du_produit"]
            site = row["Code_site"]
            
            ligne_prix_precedent = df_mois_precedent[(df_mois_precedent["Libelle_du_produit"] == produit) & (df_mois_precedent["Code_site"] == site)]
            if not ligne_prix_precedent.empty:
                prix_precedent = ligne_prix_precedent["Prix_du_produit"].values[0]
                variete = ligne_prix_precedent["Type variété (HE, O1,O2,O3)"].values[0]
                
                # Filtrer les produits de la même variété dans le fichier journalier
                produits_meme_variete = df_journalier[(df_journalier["Type variété (HE, O1,O2,O3)"] == variete) & (df_journalier["Code_site"] == site)]
                # Exclure les produits avec des prix manquants
                produits_meme_variete = produits_meme_variete.dropna(subset=["Prix_du_produit"])
                
                if not produits_meme_variete.empty:
                    # Calculer les variations individuelles de chaque produit de la même variété
                    variations = []
                    for _, ligne_meme_variete in produits_meme_variete.iterrows():
                        produit_meme_variete = ligne_meme_variete["Libelle_du_produit"]
                        ligne_prix_meme_variete_precedent = df_mois_precedent[(df_mois_precedent["Libelle_du_produit"] == produit_meme_variete) & (df_mois_precedent["Code_site"] == site)]
                        if not ligne_prix_meme_variete_precedent.empty:
                            prix_meme_variete_precedent = ligne_prix_meme_variete_precedent["Prix_du_produit"].values[0]
                            prix_actuel = ligne_meme_variete["Prix_du_produit"]
                            variation = abs(prix_actuel - prix_meme_variete_precedent)
                            variations.append(variation)
                    
                    # Calculer la variation moyenne
                    if variations:
                        variation_moyenne = sum(variations) / len(variations)
                        nouveau_prix = prix_precedent * variation_moyenne
                        df_journalier.at[index, "Prix_du_produit"] = nouveau_prix
                        logging.info(f"Imputation faite pour {produit} avec le nouveau prix {nouveau_prix}.")
                else:
                    logging.info(f"Aucun produit similaire trouvé pour la variété {variete}.")
            else:
                logging.info(f"le produit {produit} n'est pas parmi ceux du mois précédent.")

dossier_fichiers_journaliers = os.path.join("Data_Correction2", "avril")
dossier_sortie = "dossier_imputation"

if not os.path.exists(dossier_sortie):
    os.makedirs(dossier_sortie)

logging.info("Début du traitement des fichiers journaliers.")

for nom_fichier in os.listdir(dossier_fichiers_journaliers):
    if nom_fichier.endswith(".xlsx"):
        chemin_fichier_journalier = os.path.join(dossier_fichiers_journaliers, nom_fichier)
        df_journalier = pd.read_excel(chemin_fichier_journalier)

        logging.info(f"Traitement du fichier {nom_fichier}.")

        imputer_prix_manquants(df_journalier, df_mois_precedent)
        
        chemin_fichier_corrige = os.path.join(dossier_sortie, f"{nom_fichier}_clean.xlsx")
        df_journalier.to_excel(chemin_fichier_corrige, index=False)

logging.info("Traitement terminé.")


2024-06-12 15:41:18,512 - INFO - Fichier agrégé du mois précédent chargé.
2024-06-12 15:41:18,512 - INFO - Début du traitement des fichiers journaliers.
2024-06-12 15:41:19,197 - INFO - Traitement du fichier Data_Scrapping_02042024.xlsx.
2024-06-12 15:41:19,804 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 122250.00090609114.
2024-06-12 15:41:20,326 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 695760.5239291317.
2024-06-12 15:41:20,879 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 4071231.5494635776.
2024-06-12 15:41:21,514 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 23921901.975018717.
2024-06-12 15:41:22,146 - INFO - Imputation faite pour Beurre doux gastronomique avec le nouveau prix 140566378.04635423.
2024-06-12 15:41:22,760 - INFO - Imputation faite pour Beurre doux avec le nouveau prix 571893466.9034072.
2024-06-12 15:41:23,356 - INFO - Imputation fa

##### CALCUL DE L'INDICE

##### L’indice de DUTOT (pour les variétés homogènes)

In [ ]:
# Chemins vers des fichiers correspondants
#fichier_agrege_mois = os.path.join("Data_Correction2", "avril", "agregation_avril.xlsx")
#fichier_prix_reference = os.path.join("Data_Correction2", "prix_reference.xlsx")

df_agrege_mois = pd.read_excel(fichier_agrege_mois)
df_prix_reference = pd.read_excel(fichier_prix_reference)

#Calculer la moyenne des prix pour chaque variété pour le mois en cours
#moyennes_prix_variete = df_agrege_mois.groupby(["Type variété (HE, O1,O2,O3)", "Code_site"])["Prix_du_produit"].mean().reset_index()
moyennes_prix_variete = df_agrege_mois.groupby(["Type variété (HE, O1,O2,O3)", "Code_site"]).agg({'Prix_du_produit': 'mean'}).reset_index()
somme_prix_variete = moyennes_prix_variete['Prix_du_produit'].mean()

# Calculer la somme des prix de référence pour chaque variété
somme_prix_reference = df_prix_reference.groupby("Type variété (HE, O1,O2,O3)")["Prix_reference"].mean()

nombre_sites = df_agrege_mois["Code_site"].nunique()

indices_variete = moyennes_prix_variete / somme_prix_reference



###